In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 79 kB 7.3 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=5db6d00b3eefe661e75b6bdd8d0352af492a7a605435a38003fbf789b94864b3
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Folium installed
Libraries imported.


All required libraries are downloaded and imported for use on the notebook. Next step is to download the New York neighbourhoods data.

In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
#newyork_data

--------------------------------------------
Now for the import of the data
----------------------------------------------

In [4]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude','Restaurants'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon,
                                         'Restaurants': 0}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude,Restaurants
0,Bronx,Wakefield,40.894705,-73.847201,0
1,Bronx,Co-op City,40.874294,-73.829939,0
2,Bronx,Eastchester,40.887556,-73.827806,0
3,Bronx,Fieldston,40.895437,-73.905643,0
4,Bronx,Riverdale,40.890834,-73.912585,0


In [5]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


For better depiction of the neighbourhoods of New York, map visualisation is done with Folium.

In [6]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Now, for the restaurant data, Foursquare API is used.
First, credentials need to be set up.

In [7]:
CLIENT_ID = 'WKU5FJQS3TB3SZOHS4AWSX1QWLUCN5AKPV5Y0PTK14ANCI5E' # your Foursquare ID
CLIENT_SECRET = 'GKC01I025POJ5KNXQMQ3AQLYWEXTP3JUOTWEIDBHF4UAGHSF' # your Foursquare Secret
ACCESS_TOKEN = 'MMWM54MGF5KRNG20UR5URJCJSEYAOO0S2VQU0XII2AKT4APC' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 10000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WKU5FJQS3TB3SZOHS4AWSX1QWLUCN5AKPV5Y0PTK14ANCI5E
CLIENT_SECRET:GKC01I025POJ5KNXQMQ3AQLYWEXTP3JUOTWEIDBHF4UAGHSF


In [8]:
search_query = 'Italian'
radius = 30000
print(search_query + ' .... OK!')

Italian .... OK!


In [9]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,latitude, longitude, ACCESS_TOKEN, VERSION, search_query,radius, LIMIT)
results = requests.get(url).json()
#results

In [12]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)           
    
print("Number of Italian restaurants: "+ str(dataframe.shape[0]))

Number of Italian restaurants: 50


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [13]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Harry's Italian Pizza Bar,Italian Restaurant,2 Gold St,at Maiden Ln.,40.707634,-74.006996,"[{'label': 'display', 'lat': 40.7076335455641,...",573,10038,US,New York,NY,United States,"[2 Gold St (at Maiden Ln.), New York, NY 10038...",NaN,4a4560a7f964a5201aa81fe3
1,Italian American Museum Of New York,History Museum,155 Mulberry St,at Grand St,40.719191,-73.997376,"[{'label': 'display', 'lat': 40.71919065572268...",1024,10013,US,New York,NY,United States,"[155 Mulberry St (at Grand St), New York, NY 1...",NaN,4b9d4c67f964a5206ca136e3
2,Faicco's Italian Specialties,Sandwich Place,260 Bleecker St,at Cornelia St,40.731117,-74.003043,"[{'label': 'display', 'lat': 40.73111661087598...",2062,10014,US,New York,NY,United States,"[260 Bleecker St (at Cornelia St), New York, N...",NaN,4a74a36af964a520fede1fe3
3,Pisillo Italian Panini,Sandwich Place,97 Nassau St,at Ann St,40.710530,-74.007526,"[{'label': 'display', 'lat': 40.71053008446078...",275,10038,US,New York,NY,United States,"[97 Nassau St (at Ann St), New York, NY 10038,...",NaN,528bf16711d2b7722da6b51c
4,Carmela's Italian Specialties,Pizza Place,30 Water St,NaN,40.703293,-74.011017,"[{'label': 'display', 'lat': 40.70329284667969...",1131,10004,US,New York,NJ,United States,"[30 Water St, New York, NJ 10004, United States]",NaN,4e4c4e62bd413c4cc6686059
5,Pisillo Italian Cafe,Café,NaN,NaN,40.710493,-74.007546,"[{'label': 'display', 'lat': 40.71049280792626...",280,NaN,US,New York,NY,United States,"[New York, NY, United States]",NaN,588a4316326c5a4b60559f17
6,Tableside Italian Cook Shoppe,Italian Restaurant,345 E 6th St,1st and 2nd Ave,40.726556,-73.986514,"[{'label': 'display', 'lat': 40.72655617438330...",2253,10003,US,New York,NY,United States,"[345 E 6th St (1st and 2nd Ave), New York, NY ...",NaN,58dd8ac3a8b75966fda9a547
7,Losurdo Brothers / Tony's Italian Bakery & Deli,Bakery,410 2nd St,btw Grand St & Adams St,40.740126,-74.036964,"[{'label': 'display', 'lat': 40.74012586093679...",4014,07030,US,Hoboken,NJ,United States,"[410 2nd St (btw Grand St & Adams St), Hoboken...",NaN,4b269f5af964a520817e24e3
8,Caruso's Italian Restaurant,Italian Restaurant,42 Broadway,NaN,40.710509,-74.008276,"[{'label': 'display', 'lat': 40.71050909999999...",312,10004,US,New York,NY,United States,"[42 Broadway, New York, NY 10004, United States]",NaN,4e4c4d28bd413c4cc66857a5
9,Benvenuti Italian Specialties,Gourmet Shop,235 S End Ave,NaN,40.709681,-74.016520,"[{'label': 'display', 'lat': 40.70968105555279...",949,10280,US,New York,NY,United States,"[235 S End Ave, New York, NY 10280, United Sta...",NaN,4f3feae2e4b0ac9416a72118


In [14]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Now for the clustering of these restaurants, DBScan is used.

In [15]:
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 
#!conda install -c conda-forge  basemap matplotlib==3.1 -y
from sklearn.cluster import DBSCAN
print('Libraries imported.')

Libraries imported.


In [16]:
epsilon = 10
minimumSamples = 10

X=[]
#Y=[]

for lat, lng, in zip(dataframe_filtered.lat, dataframe_filtered.lng):
    X.append([lat,lng])
#for late, lon in zip(neighborhoods.latitude,neighborhoods.longitude):
#    Y.append()
X = np.array(X)
db = DBSCAN(eps=epsilon, min_samples=minimumSamples).fit(X)
labels = db.labels_
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

---------------------------------------------------------
K-Means Algorithm Will Be Used
---------------------------------------------------------

In [20]:
from sklearn.cluster import KMeans 

In [21]:
colors_map = np.array(['b', 'r'])
def assign_members(x1, x2, centers):
    compare_to_first_center = np.sqrt(np.square(np.array(x1) - centers[0][0]) + np.square(np.array(x2) - centers[0][1]))
    compare_to_second_center = np.sqrt(np.square(np.array(x1) - centers[1][0]) + np.square(np.array(x2) - centers[1][1]))
    class_of_points = compare_to_first_center > compare_to_second_center
    colors = colors_map[class_of_points + 1 - 1]
    return colors, class_of_points

print('assign_members function defined!')

assign_members function defined!


In [22]:
# update means
def update_centers(x1, x2, class_of_points):
    center1 = [np.mean(np.array(x1)[~class_of_points]), np.mean(np.array(x2)[~class_of_points])]
    center2 = [np.mean(np.array(x1)[class_of_points]), np.mean(np.array(x2)[class_of_points])]
    return [center1, center2]

print('assign_members function defined!')

assign_members function defined!


In [23]:
k_means = KMeans(init="k-means++", n_clusters=6, n_init=12)
k_means.fit(X)
k_means_labels = k_means.labels_
k_means_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 3, 4, 0, 1, 1, 0, 3, 2, 0, 3, 1, 1, 0, 0, 1, 1, 1, 4, 1, 3, 1,
       1, 1, 2, 5, 4, 3], dtype=int32)

In [24]:
k_means_cluster_centers = k_means.cluster_centers_
k_means_cluster_centers

array([[ 40.71575486, -74.00830764],
       [ 40.7636987 , -73.97833418],
       [ 40.74489205, -73.86798259],
       [ 40.74429849, -73.92554598],
       [ 40.79448892, -74.01656678],
       [ 40.615195  , -73.994556  ]])

In [92]:
clus_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel
ind=0

# add the Italian restaurants as blue circle markers
for i in k_means_cluster_centers:
    folium.CircleMarker(
        [k_means_cluster_centers[ind][0], k_means_cluster_centers[ind][1]],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(clus_map)
    ind=ind+1
    
folium.CircleMarker(
        [k_means_cluster_centers[5][0], k_means_cluster_centers[5][1]],
        radius=20,
        color='red',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(clus_map)
# display map
clus_map